# Image Stitching 
<img src="images\stitched.png" width=500 />

In this lab, we are going to: 
- Detect feature points in images.
- Calculate descriptors for every keypoint in images using SIFT.
- Compute feature-space distances between every pair of feature points from image source and destination.
- Select good matches (using Lowe's ratio test) based on the distance matrix above.
- Get the homography matrix using the RANSAC algorithm.
- Generate the panorama from the images.

## Let's understand the stitching algorithm

In the beginning, we have two images: the **destination image** (left side) and the **source image** (right side). 

We want to "transform" the **source image** with respect to the **destination image**, such that the two images will be well aligned. We accomplish this by applying a perspective transformation to the **source image**. We obtain the perspective transformation based on the matches between the two images.
<table width="950px">
<tr>
<th><center>Destination</center></th>
<th><center>Source</center></th> 
</tr>
<tr>
<td><img src="images\dest.jfif" width=350 /></td>
<td> <img src="images\source.jfif" width=350 /></td> 
</tr>
</table>   

Now, we made the **source image** to have the same persective as the **destination image**.
<table width="950px">
<tr>
<th><center>Destination</center></th>
<th><center>Result Source</center></th> 
</tr>
<tr>
<td><img src="images\dest.jfif" width=350 /></td>
<td> <img src="images\result.png" width=510 /></td> 
</tr>
</table>
    
And in the end, we copy the destination image in the **result source** image.
<img src="images\stitched_2.png" width=510 />

In [2]:
import sys 
import cv2 as cv
import typing as ty
import pdb
import numpy as np
import os
import copy

In [3]:
def show_image(image_, window_name='image', timeout=0):
    """
    Show image.
    :param image_
    :param window_name
    :param timeout
    """
    cv.imshow(window_name, np.uint8(image_))
    cv.waitKey(timeout)
    cv.destroyAllWindows()

In [4]:
def get_keypoints_and_features(image, show_details=False) -> tuple:
    """
    TODO:
    1. Convert the image to grayscale.
    2. Create the SIFT object. (https://docs.opencv.org/master/da/df5/tutorial_py_sift_intro.html)
    3. Find keypoints from the grayscale image.
    4. Compute the features based on the grayscale image and the keypoints.

    :param image.
    :return the keypoints: [cv.Keypoint] and the features: np.ndarray for each keypoint.
    """

    def show_keypoints(image_, keypoints_):
        """
        Show the keypoints found in the image.
        """
        image_output = image_.copy()
        image_output = cv.drawKeypoints(
            image, keypoints_, image_output, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        show_image(image_output, 'keypoints')

    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    sift = cv.xfeatures2d.SIFT_create()
    keypoints, features = sift.detectAndCompute(gray_image, None)

    if show_details:
        show_keypoints(image, keypoints)

    return keypoints, features


In [5]:
img_test = cv.imread('data/stitches/2/2.jfif')
k1, f1 = get_keypoints_and_features(img_test, show_details=False)
img_test2 = cv.imread('data/stitches/2/3.jfif')
k2, f2 = get_keypoints_and_features(img_test2, show_details=False)

[ WARN:0@1.905] global shadow_sift.hpp:13 SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


In [6]:
def match_features(features_source, features_dest) -> ty.List[ty.List[cv.DMatch]]:
    """
    Match features from the source image with the features from the destination image.
    :return: [[DMatch]] - The rusult of the matching. For each set of features from the source image,
    it returns the first 'K' matchings from the destination images.
    """
 
    feature_matcher = cv.DescriptorMatcher_create("FlannBased")
    matches = feature_matcher.knnMatch(features_source, features_dest, k=2)   
    return matches

In [7]:
matched_features = match_features(f1, f2)

In [8]:
matched_features[0][0].distance, matched_features[0][1].distance, matched_features[0][0]

(286.7542419433594, 362.1187744140625, < cv2.DMatch 0x7fa9abce3e90>)

In [9]:
import typing as ty


def generate_homography(all_matches:  ty.List[cv.DMatch], keypoints_source: ty.List[cv.KeyPoint], keypoints_dest: ty.List[cv.KeyPoint],
                        ratio: float = 0.75, ransac_rep: int = 4.0):
    """
    TODO:
    1. Find the matchings that pass the Lowe's ratio test (ratio parameter).
    2. Get the coordinates of the keypoints from the source image.
    3. Get the coordinates of the keypoints from the destination image.
    4. Obtain the Homagraphy. (https://docs.opencv.org/master/d1/de0/tutorial_py_feature_homography.html)
    :param all_matches [DMatch]
    :param keypoints_source [cv.Point]
    :param ratio - Lowe's ratio test (the ratio 1st neighbour distance / 2nd neighbour distance)
    :param keypoints_source: nd.array [Nx2] (x, y coordinates)
    :param keypoints_dest: nd.array [Nx2] (x, y coordinates)
    :param ransac_rep: float. The threshold in the RANSAC algorithm.
    :return: The homography matrix.

    class DMatch:
        distance - Distance between descriptors. The lower, the better it is.
        imgIdx - Index of the train image
        queryIdx - Index of the descriptor in query descriptors
        trainIdx - Index of the descriptor in train descriptors

    class KeyPoint:
        pt - The x, y coordinates of a point.

    """
    if not all_matches:
        return None
    matches: ty.List[cv.DMatch] = []
    for match in all_matches:
        if len(match) == 2 and match[0].distance / match[1].distance < ratio:
            matches.append(match[0])
    points_source = np.float32([keypoints_source[m.queryIdx].pt for m in matches])
    points_destin = np.float32([keypoints_dest[m.trainIdx].pt for m in matches])
    if len(points_source) > 4:
        H, _ = cv.findHomography(
            points_source, points_destin, cv.RANSAC, ransac_rep)
        return H


In [10]:
H = generate_homography(matched_features, k1, k2)
H

array([[ 1.11373056e+00, -1.41136727e-02, -1.51503046e+02],
       [ 6.04398631e-02,  1.06568490e+00,  8.29887469e+00],
       [ 3.10572413e-04, -5.42831673e-05,  1.00000000e+00]])

In [25]:
def stitch_images(image_source, image_dest, show_details=False):
    """ 
    :param image_source (image from the right part).
    :param image_dest (image from the left part).
    :param show_details
    :return - the stitched image.
    TODO:
    1. Get the keypoints and the features from the source image.
    2. Get the keypoints and the features from the destination image.
    3. Match the features.
    4. Find the homography matrix.
    5. Apply the homography matrix on the source image.
    (https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_geometric_transformations/py_geometric_transformations.html#affine-transformation)
    6. Copy the destination image in the resulting image from the previous point.
    """
    keypoints_source, features_source = get_keypoints_and_features(image_source, show_details=show_details)
    keypoints_dest, features_dest = get_keypoints_and_features(image_dest, show_details=show_details)

    def show_matches(all_matches_, n=10):
        matches = sorted(all_matches_, key = lambda x:x[0].distance)
        matches = matches[:n] 
        image_output = cv.drawMatchesKnn(image_source, keypoints_source,
                                         image_dest, keypoints_dest,
                                         matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        show_image(image_output, 'matches')

    # TODO:
    all_matches = match_features(features_source, features_dest)
    if show_details:
        show_matches(all_matches_=all_matches)
    H = generate_homography(all_matches, keypoints_source, keypoints_dest)

    if show_details:
        show_matches(copy.copy(all_matches))
    
    result = cv.warpPerspective(image_source, H, (image_dest.shape[1] + image_source.shape[1], image_dest.shape[0]))
    result[:image_dest.shape[0], :image_dest.shape[1]] = image_dest
    return result

In [32]:
img_test = cv.imread('data/stitches/13/1.jpg')
img_test2 = cv.imread('data/stitches/13/2.jpg')
stitched_image = stitch_images(img_test2, img_test, show_details=False)
show_image(stitched_image)

In [26]:
def pad_image(image_, procent=0.1):
    pad_h = int(image_.shape[0] * procent)
    pad_w = int(image_.shape[1] * procent)
    big_image = np.zeros((image_.shape[0] + 2 * pad_h, image_.shape[1] + pad_w, 3), np.uint8)
    big_image[pad_h: pad_h + image_.shape[0], pad_w: pad_w + image_.shape[1]] = image_.copy()
    return big_image

In [13]:
def stitch_images_from_folder(folder_path, show_details=False):
    """
    Stitch the images from the last image to the first.
    TODO:
    1. Read the images from the folder, sort them (ascending order), 
    then reverse the list (because we are going to stitch them from the last image to the first).
    2. Read the first image (the source image).
    3. While you have unread images, read the next image (destination image), 
    stitch it with the source image then save the resulting image in the source image (in the next step it will be the source image).
    """
    image_names = os.listdir(folder_path)
    image_names.sort()
    image_names = image_names[::-1]
    
    assert len(image_names) >= 2
    result = ... 
        
    return result
    

In [14]:
stitched = stitch_images_from_folder('data/stitches/2', show_details=True)
show_image(stitched)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'ellipsis'

In [ ]:
# stitch all available images
base_folder = 'data/stitches'
folder_names = os.listdir(base_folder)
for folder_name in folder_names:
    stitched = stitch_images_from_folder(folder_path=os.path.join(base_folder, folder_name))
    stitched = cv.resize(stitched, None, fx=0.5, fy=0.5)  # use this only if you have a small screen
    show_image(stitched)

## Stitch image Now and Then 
In this scenario, we are not going to merge (stitched) the images, but to put the content of the source image where it belongs in the destination image.

In the left side, we have the "*now*" image (**destination**) and in the right part we have the "*then*" image (**source**).


<table width="950px">
<tr>
<th><center>Now</center></th>
<th><center>Then</center></th> 
</tr>
<tr>
<td><img src="images\2now.png" width=350 allign="left"/></td>
<td> <img src="images\2then.png" width=250 /></td> 
</tr>
</table>   

Now, we made the ***then* image** to have the same perspective as the ***now* image**.
<table width="950px">
<tr>
<th><center>Now</center></th>
<th><center>Result then</center></th> 
</tr>
<tr>
<td><img src="images\2now.png" width=350 /></td>
<td> <img src="images\result_source.png" width=350 /></td> 
</tr>
</table>
    
And in the end, we copy the destination image in the **result then** image, but without replacing the pixels that are already occupied by the *then* image.
<img src="images\result_now_then.png" width=350 />

In [38]:
def stitch_images_inside(image_source, image_dest, show_details=False):
    """ 
    :param image_source (image from the right part).
    :param image_dest (image from the lest part).
    :param show_details
    :return: the stitched image.
    TODO:
    1. Get the keypoints and the features from the source image.
    2. Get the keypoints and the features from the destination image.
    3. Match the features.
    4. Find the homography matrix.
    5. Apply the homography matrix on the source image.
    6. Copy the destination image in the resulting image from the previous point, but keep the resulting pixels in place!
    """
    keypoints_source, features_source = get_keypoints_and_features(image_source, show_details=show_details)
    keypoints_dest, features_dest = get_keypoints_and_features(image_dest, show_details=show_details)
 
    def show_matches(all_matches_, n=10):
        matches = sorted(all_matches_, key = lambda x:x[0].distance)
        matches = matches[:n] 
        image_output = cv.drawMatchesKnn(image_source, keypoints_source, 
                                         image_dest, keypoints_dest,  
                                         matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        show_image(image_output, 'matches')
 

    # TODO:
    all_matches = match_features(features_source, features_dest)
    if show_details:
        show_matches(all_matches_=all_matches)
    H = generate_homography(all_matches, keypoints_source, keypoints_dest)

    if show_details:
        show_matches(copy.copy(all_matches))
    
    result = cv.warpPerspective(image_source, H, (image_dest.shape[1], image_dest.shape[0]))
    mask = result[:, :] == np.array([0])
    result = result * (1 - mask) + image_dest * mask
    return result
        

In [34]:
image_now = cv.imread('data/nowthen/2now.png')
image_then = cv.imread('data/nowthen/2then.png')

In [37]:
stitched = stitch_images_inside(image_source=image_then, image_dest=image_now, show_details=True)
show_image(stitched) 

In [ ]:
# stitch all available now/then images
base_folder = 'data/nowthen'
image_names = os.listdir(base_folder)
num_images = len(image_names) // 2
for i in range(1, num_images + 1):
    image_now = cv.imread(os.path.join(base_folder, f'{i}now.png'))
    image_then = cv.imread(os.path.join(base_folder, f'{i}then.png'))
    stitched = stitch_images_inside(image_source=image_then, image_dest=image_now, show_details=False)
    show_image(stitched)